In [278]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [279]:
url = 'https://mars.nasa.gov/news'

In [280]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [281]:
title_results = soup.find('div', class_="content_title")
news_title = title_results.get_text(strip=True)


In [282]:
para_results = soup.find('div', class_="rollover_description_inner")
news_p = para_results.get_text(strip=True)

In [283]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

browser.find_by_css('div[class="NavDesktopDropdown -active"]')[0].click()
browser.click_link_by_partial_text('Featured Image')

import time 
time.sleep(3)

html = browser.html
soup = bs(html, 'html.parser')
main = soup.find('main')
img = main.find('img')
featured_image_url = img['src']






[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jeremykalmus/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache
 


In [284]:
url = 'https://space-facts.com/mars/'
mars_data_table = pd.read_html(url, header=None)
mars_data_table = mars_data_table[0]
mars_data_table = mars_data_table.rename(columns={0:'Description',1: 'Mars'})
mars_data_html = mars_data_table.to_html()


In [285]:

url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

url_list = []
hemi_list = []

items = soup.find_all('div', class_='description')

for item in items:
    a = item.find('a', class_='itemLink')
    hemi = a.text.strip()
    hemi_list.append(hemi)
    url = item.find('a')['href']
    url_list.append(url)


hemi_list = [i.split(' Enhanced',1)[0] for i in hemi_list]

orignial_img_list = []

for url in url_list:
    browser.visit(f"https://astrogeology.usgs.gov/{url}")
    html = browser.html
    soup = bs(html, 'html.parser')
    downloads = soup.find_all('li')
    temp_list = []
    for download in downloads:
        orignial_img = download.find('a')['href']
        temp_list.append(orignial_img)
    
    orignial_img_list.append(temp_list[1]) 

hemisphere_image_urls = []




browser.quit()

for i in range(0,4,1):
    temp_dict = {'title': hemi_list[i], 'img_url': orignial_img_list[i]}
    hemisphere_image_urls.append(temp_dict)

In [286]:
mars_dict = ({'Recent_News':news_title, 'News_Story':news_p }, {'Featured_Image':featured_image_url}, {'Mars_Data': mars_data_html}, {'Mars_Hemispheres': hemisphere_image_urls} )

In [287]:
    conn = "mongodb://localhost:27017"
    client = pymongo.MongoClient(conn)
    db = client.mars_db 
    mars_collection = db.mars_collection

    mars_collection.insert_many(
        mars_dict
    )

In [288]:
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.width-1024.jpg'

In [289]:
print(mars_dict[2])

{'Mars_Data': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2n